# Phân tích tập dữ liệu các trận đá bóng ở châu Âu giai đoạn 2008 - 2016

Dataset dữ liệu được lấy từ [Kaggle](https://www.kaggle.com/datasets/abdelrhmanragab/european-soccer-database) gồm các thông tin của các trận đấu, cầu thủ, đội bóng ở các nước châu Âu trong 11 giải đấu khác nhau với hơn 10 ngàn trận đấu giai đoạn 2008 - 2016. Bộ dataset này có 7 file `*.csv` khác nhau chứa các thông tin:

* `Country.csv` Các quốc gia được khảo sát
* `League.csv` Các giải đấu được khảo sát
* `Match.csv` Các trận đấu diễn tra trong giai đoạn 2008 - 2016
* `Player.csv` Thông tin các cầu thủ (tên, ngày sinh, chiều cao, cân nặng)
* `Player_Attributes.csv` Thông tin đặc tính cầu thủ
* `Team.csv` Thông tin đội bóng
* `Team_Attributes.csv` Thông tin đặc tính của đội bóng

Với tập dữ liệu này, thông qua việc khảo sát phân tích sẽ cố gắng tìm câu trả lời cho các câu hỏi sau

* Có bao nhiêu trận đấu trong mỗi giải đấu ở mùa giải 2016?
* Giải đấu nào có số trận hòa nhiều nhất trong mùa giải 2016?
* Giải đấu nào có số trận không hòa nhiều nhất trong mùa giải 2016?
* Đội bóng nào để thua ít trận nhất trong mùa giải 2016?
* Đội bóng nào có số trận thắng nhiều nhất trong mùa giải 2016?
* Giải đấu nào có nhiều bàn thắng nhất trong mùa giải 2016?
* Đội bóng nào có kết quả cải thiện tốt nhất qua thời gian?
* Cầu thủ nào sút phạt nhiều nhất?
* Có bao nhiêu cầu thủ có hiệu suất tổng thể hơn 90?

# Chuẩn bị thư viện

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Khảo sát dữ liệu

Nạp dữ liệu vào bộ nhớ

In [2]:
df_match = pd.read_csv('./data/Match.csv')
df_player_attribute = pd.read_csv('./data/Player_Attributes.csv')
df_team_attribute = pd.read_csv('./data/Team_Attributes.csv')
df_team = pd.read_csv('./data/Team.csv')
df_player = pd.read_csv('./data/Player.csv')
df_country = pd.read_csv('./data/Country.csv')
df_league = pd.read_csv('./data/League.csv')

## Dữ liệu trận đấu

Lấy mẫu tập dữ liệu trận đấu

In [3]:
df_match.head() 

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


Các cột trong bảng trận đấu

In [4]:
df_match.columns 

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

Số lượng dòng và cột

In [5]:
df_match.shape

(25979, 115)

Thông tin data frame

In [6]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Columns: 115 entries, id to BSA
dtypes: float64(96), int64(9), object(10)
memory usage: 22.8+ MB


Số lượng dữ liệu bị trùng

In [7]:
df_match.duplicated().sum()

0

Tổng số dữ liệu `NULL`

In [8]:
df_match.isna().sum().sum()

407395

Thống kê dữ liệu

In [9]:
df_match.describe()

,id,country_id,league_id,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
count,25979.000000,25979.000000,25979.000000,25979.000000,2.597900e+04,25979.000000,25979.000000,25979.000000,25979.000000,24158.000000,...,17097.000000,22568.000000,22568.000000,22568.000000,14162.000000,14162.000000,14162.000000,14161.000000,14161.000000,14161.000000
mean,12990.000000,11738.630317,11738.630317,18.242773,1.195429e+06,9984.371993,9984.475115,1.544594,1.160938,0.999586,...,4.622343,2.668107,3.899048,4.840281,2.498764,3.648189,4.353097,2.497894,3.660742,4.405663
std,7499.635658,7553.936759,7553.936759,10.407354,4.946279e+05,14087.453758,14087.445135,1.297158,1.142110,0.022284,...,3.632164,1.928753,1.248221,4.318338,1.489299,0.867440,3.010189,1.507793,0.868272,3.189814
min,1.000000,1.000000,1.000000,1.000000,4.831290e+05,1601.000000,1601.000000,0.000000,0.000000,0.000000,...,1.100000,1.030000,1.620000,1.080000,1.050000,1.450000,1.120000,1.040000,1.330000,1.120000
25%,6495.500000,4769.000000,4769.000000,9.000000,7.684365e+05,8475.000000,8475.000000,1.000000,0.000000,1.000000,...,2.500000,1.700000,3.300000,2.550000,1.670000,3.200000,2.500000,1.670000,3.250000,2.500000
50%,12990.000000,10257.000000,10257.000000,18.000000,1.147511e+06,8697.000000,8697.000000,1.000000,1.000000,1.000000,...,3.500000,2.150000,3.500000,3.500000,2.100000,3.300000,3.400000,2.100000,3.400000,3.400000
75%,19484.500000,17642.000000,17642.000000,27.000000,1.709852e+06,9925.000000,9925.000000,2.000000,2.000000,1.000000,...,5.250000,2.800000,4.000000,5.400000,2.650000,3.750000,5.000000,2.620000,3.750000,5.000000
max,25979.000000,24558.000000,24558.000000,38.000000,2.216672e+06,274581.000000,274581.000000,10.000000,9.000000,2.000000,...,41.000000,36.000000,26.000000,67.000000,21.000000,11.000000,34.000000,17.000000,13.000000,34.000000


**Kết luận**

Dữ liệu gồm có 25979 dòng và 115 cột, không có dòng bị lặp.

Dữ liệu có số lượng giá trị `NULL` khá nhiều (407395) tuy nhiên các giá trị này nằm ở các cột không xét nên sẽ được lược bỏ.

## Dữ liệu quốc gia

Lấy mẫu dữ liệu

In [10]:
df_country

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


Kích thước dữ liệu

In [11]:
df_country.shape

(11, 2)

Thông tin bảng dữ liệu

In [12]:
df_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11 non-null     int64 
 1   name    11 non-null     object
dtypes: int64(1), object(1)
memory usage: 304.0+ bytes


Thống kê dữ liệu

In [13]:
df_country.describe()

,id
count,11.000000
mean,12452.090909
std,8215.308472
min,1.000000
25%,6289.000000
50%,13274.000000
75%,18668.000000
max,24558.000000


**Kết luận**

Bảng dữ liệu quốc gia có 11 dòng và 2 cột, không có dữ liệu trùng lắp và không có dữ liệu `NULL`

## Dữ liệu đội bóng

Lấy mẫu dữ liệu

In [14]:
df_team.head() 

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


Kích thước dữ liệu

In [15]:
df_team.shape

(299, 5)

Thông tin bảng đội bóng

In [16]:
df_team.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                299 non-null    int64  
 1   team_api_id       299 non-null    int64  
 2   team_fifa_api_id  288 non-null    float64
 3   team_long_name    299 non-null    object 
 4   team_short_name   299 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 11.8+ KB


Đếm số dữ liệu trùng lắp

In [17]:
df_team.duplicated().sum()

0

Đếm giá trị `NULL` theo cột

In [18]:
df_team.isna().sum()

id                   0
team_api_id          0
team_fifa_api_id    11
team_long_name       0
team_short_name      0
dtype: int64

Thống kê dữ liệu

In [19]:
df_team.describe()

,id,team_api_id,team_fifa_api_id
count,299.000000,299.000000,288.000000
mean,23735.301003,12340.521739,21534.305556
std,15167.914719,25940.411135,42456.439408
min,1.000000,1601.000000,1.000000
25%,9552.500000,8349.000000,178.750000
50%,22805.000000,8655.000000,673.500000
75%,36250.500000,9886.500000,1910.750000
max,51606.000000,274581.000000,112513.000000


**Kết luận**

Dữ liệu đội bóng gồm có 299 dòng và 5 cột, không có dữ liệu trùng, có 11 giá trị `NULL` tuy nhiên đều nằm ở cột `team_fifa_api_id`.

## Dữ liệu đặc tính đội bóng

Lấy mẫu dữ liệu

In [20]:
df_team_attribute.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


Kích thước dữ liệu

In [21]:
df_team_attribute.shape

(1458, 25)

Thông tin bảng đặc tính đội bóng

In [22]:
df_team_attribute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              1458 non-null   int64  
 1   team_fifa_api_id                1458 non-null   int64  
 2   team_api_id                     1458 non-null   int64  
 3   date                            1458 non-null   object 
 4   buildUpPlaySpeed                1458 non-null   int64  
 5   buildUpPlaySpeedClass           1458 non-null   object 
 6   buildUpPlayDribbling            489 non-null    float64
 7   buildUpPlayDribblingClass       1458 non-null   object 
 8   buildUpPlayPassing              1458 non-null   int64  
 9   buildUpPlayPassingClass         1458 non-null   object 
 10  buildUpPlayPositioningClass     1458 non-null   object 
 11  chanceCreationPassing           1458 non-null   int64  
 12  chanceCreationPassingClass      14

Số lượng dữ liệu bị trùng ở các cột

In [23]:
df_team_attribute.isna().sum()

id                                  0
team_fifa_api_id                    0
team_api_id                         0
date                                0
buildUpPlaySpeed                    0
buildUpPlaySpeedClass               0
buildUpPlayDribbling              969
buildUpPlayDribblingClass           0
buildUpPlayPassing                  0
buildUpPlayPassingClass             0
buildUpPlayPositioningClass         0
chanceCreationPassing               0
chanceCreationPassingClass          0
chanceCreationCrossing              0
chanceCreationCrossingClass         0
chanceCreationShooting              0
chanceCreationShootingClass         0
chanceCreationPositioningClass      0
defencePressure                     0
defencePressureClass                0
defenceAggression                   0
defenceAggressionClass              0
defenceTeamWidth                    0
defenceTeamWidthClass               0
defenceDefenderLineClass            0
dtype: int64

Thống kê dữ liệu

In [24]:
df_team_attribute.describe()

,id,team_fifa_api_id,team_api_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
count,1458.000000,1458.000000,1458.000000,1458.000000,489.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000
mean,729.500000,17706.982167,9995.727023,52.462277,48.607362,48.490398,52.165295,53.731824,53.969136,46.017147,49.251029,52.185871
std,421.032659,39179.857739,13264.869900,11.545869,9.678290,10.896101,10.360793,11.086796,10.327566,10.227225,9.738028,9.574712
min,1.000000,1.000000,1601.000000,20.000000,24.000000,20.000000,21.000000,20.000000,22.000000,23.000000,24.000000,29.000000
25%,365.250000,110.000000,8457.750000,45.000000,42.000000,40.000000,46.000000,47.000000,48.000000,39.000000,44.000000,47.000000
50%,729.500000,485.000000,8674.000000,52.000000,49.000000,50.000000,52.000000,53.000000,53.000000,45.000000,48.000000,52.000000
75%,1093.750000,1900.000000,9904.000000,62.000000,55.000000,55.000000,59.000000,62.000000,61.000000,51.000000,55.000000,58.000000
max,1458.000000,112513.000000,274581.000000,80.000000,77.000000,80.000000,80.000000,80.000000,80.000000,72.000000,72.000000,73.000000


**Kết luận**

Tập dữ liệu có 1458 dòng và 25 cột, không có dữ liệu trùng, nhiều dữ liệu bị mất ở cột `buildUpPlayDribbling`.

## Dữ liệu giải đấu

Lấy mẫu dữ liệu

In [25]:
df_league

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


Kích thước dữ liệu

In [26]:
df_league.shape

(11, 3)

Thống kê dữ liệu

In [27]:
df_league.describe()

,id,country_id
count,11.000000,11.000000
mean,12452.090909,12452.090909
std,8215.308472,8215.308472
min,1.000000,1.000000
25%,6289.000000,6289.000000
50%,13274.000000,13274.000000
75%,18668.000000,18668.000000
max,24558.000000,24558.000000


**Kết luận**

Dữ liệu có 11 dòng và 3 cột, không có dữ liệu trùng và không có dữ liệu trống

## Dữ liệu cầu thủ

Lấy mẫu dữ liệu

In [28]:
df_player.head() 

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


Kích thước dữ liệu

In [29]:
df_player.shape

(11060, 7)

Kiểm tra dữ liệu trùng

In [30]:
df_player.duplicated().sum()

0

Kiểm tra dữ liệu rỗng

In [31]:
df_player.isna().sum()

id                    0
player_api_id         0
player_name           0
player_fifa_api_id    0
birthday              0
height                0
weight                0
dtype: int64

Thông tin bảng dữ liệu

In [32]:
df_player.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  11060 non-null  int64  
 1   player_api_id       11060 non-null  int64  
 2   player_name         11060 non-null  object 
 3   player_fifa_api_id  11060 non-null  int64  
 4   birthday            11060 non-null  object 
 5   height              11060 non-null  float64
 6   weight              11060 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 605.0+ KB


Thống kê dữ liệu

In [33]:
df_player.describe()

,id,player_api_id,player_fifa_api_id,height,weight
count,11060.000000,11060.000000,11060.000000,11060.000000,11060.000000
mean,5537.511392,156582.427215,165664.910488,181.867445,168.380289
std,3197.692647,160713.700624,58649.928360,6.369201,14.990217
min,1.000000,2625.000000,2.000000,157.480000,117.000000
25%,2767.750000,35555.500000,151889.500000,177.800000,159.000000
50%,5536.500000,96619.500000,184671.000000,182.880000,168.000000
75%,8306.250000,212470.500000,203883.250000,185.420000,179.000000
max,11075.000000,750584.000000,234141.000000,208.280000,243.000000


**Kết luận**

Dữ liệu ở bảng cầu thủ gồm có 11060 dòng và 7 cột, không có dữ liệu trùng và không có dữ liệu trống

## Dữ liệu đặc tính cầu thủ

Lấy mẫu dữ liệu

In [34]:
df_player_attribute.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


Kích thước dữ liệu

In [35]:
df_player_attribute.shape

(183978, 42)

Thông tin bảng dữ liệu

In [36]:
df_player_attribute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

Kiểm tra dữ liệu trùng

In [37]:
df_player_attribute.duplicated().sum()

0

Kiểm tra dữ liệu trống

In [38]:
df_player_attribute.isna().sum()

id                        0
player_fifa_api_id        0
player_api_id             0
date                      0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle     

In [39]:
df_player_attribute.isna().sum().sum()

47301

Thống kê dữ liệu

In [40]:
df_player_attribute.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


**Kết luận**

Dữ liệu đặc tính cầu thủ gồm 183978 dòng và 42 cột, không có dữ liệu trùng, có nhiều dữ liệu trống (47301) 

# Chuẩn bị dữ liệu

## Dữ liệu quốc gia

Thay thế cột `name` bằng cột `country_name` để dễ dàng cho việc kết bảng về sau

In [41]:
df_country.rename(columns={'name' : 'country_name', }, inplace=True)

In [42]:
df_country

,id,country_name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


## Dữ liệu giải đấu

Thay thế cột `name` bằng cột `league_name` để tiện cho việc kết bảng về sau

In [43]:
df_league.rename(columns={'name': 'league_name'}, inplace=True)
df_league

,id,country_id,league_name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


## Dữ liệu trận đấu

Với dữ liệu các trận đấu `df_match` ta chỉ giữ lại các cột: 

* id
* country_id
* league_id
* season
* stage
* date
* match_api_id
* home_team_api_id
* away_team_api_id
* home_team_goal
* away_team_goal

Phần còn lại không xét đến nên lược bỏ

In [44]:
df_match = df_match.loc[:,:'away_team_goal'] 

In [45]:
df_match

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3
...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,0
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,2
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,0
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,0


Chuyển cột `date` sang kiểu dữ liệu `DateTime` để dễ dàng trích xuất năm của trận đấu

In [46]:
df_match['date'] = pd.to_datetime(df_match['date'])

Thêm một cột dữ liệu `season_year` là năm của trận đấu

In [47]:
df_match['season_year'] = df_match['date'].dt.year

Dữ liệu sau khi điều chỉnh

In [48]:
df_match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,season_year
0,1,1,1,2008/2009,1,2008-08-17,492473,9987,9993,1,1,2008
1,2,1,1,2008/2009,1,2008-08-16,492474,10000,9994,0,0,2008
2,3,1,1,2008/2009,1,2008-08-16,492475,9984,8635,0,3,2008
3,4,1,1,2008/2009,1,2008-08-17,492476,9991,9998,5,0,2008
4,5,1,1,2008/2009,1,2008-08-16,492477,7947,9985,1,3,2008


### Kết bảng quốc gia

Kết bảng trận đấu với bảng quốc gia qua thuộc tính ngoại `country_id`

In [49]:
df_match = df_match.merge(df_country, how='inner', left_on= "country_id", right_on = "id")

df_match.head()

,id_x,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,season_year,id_y,country_name
0,1,1,1,2008/2009,1,2008-08-17,492473,9987,9993,1,1,2008,1,Belgium
1,2,1,1,2008/2009,1,2008-08-16,492474,10000,9994,0,0,2008,1,Belgium
2,3,1,1,2008/2009,1,2008-08-16,492475,9984,8635,0,3,2008,1,Belgium
3,4,1,1,2008/2009,1,2008-08-17,492476,9991,9998,5,0,2008,1,Belgium
4,5,1,1,2008/2009,1,2008-08-16,492477,7947,9985,1,3,2008,1,Belgium


Lược bỏ cột `id_y` (id của bảng quốc gia) và đổi tên cột `id_x` thành `id`

In [50]:
df_match.drop(columns=['id_y'], inplace=True)
df_match.rename(columns={'id_x' : 'id'}, inplace=True)

df_match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,season_year,country_name
0,1,1,1,2008/2009,1,2008-08-17,492473,9987,9993,1,1,2008,Belgium
1,2,1,1,2008/2009,1,2008-08-16,492474,10000,9994,0,0,2008,Belgium
2,3,1,1,2008/2009,1,2008-08-16,492475,9984,8635,0,3,2008,Belgium
3,4,1,1,2008/2009,1,2008-08-17,492476,9991,9998,5,0,2008,Belgium
4,5,1,1,2008/2009,1,2008-08-16,492477,7947,9985,1,3,2008,Belgium


### Kết bảng đội bóng

Các trận đấu sẽ được kết với đội bóng tương ứng qua khóa `home_team_api_id` và `away_team_api_id`; song song đó sẽ loại bỏ các cột dư thừa như cột `id` của đội bóng sau khi kết

In [51]:
df_match = df_match.merge(df_team, how='inner', left_on='home_team_api_id', right_on='team_api_id')
df_match.rename(columns={'team_long_name': 'home_team_name','country_name_x'  : 'country_name' , 'id_x' : 'id'}, inplace=True)

df_match.drop(columns=['home_team_api_id', 'team_api_id', 'id_y'], axis=1, inplace=True)

df_match.head()

,id,country_id,league_id,season,stage,date,match_api_id,away_team_api_id,home_team_goal,away_team_goal,season_year,country_name,team_fifa_api_id,home_team_name,team_short_name
0,1,1,1,2008/2009,1,2008-08-17,492473,9993,1,1,2008,Belgium,673.0,KRC Genk,GEN
1,29,1,1,2008/2009,12,2008-11-15,492583,9999,1,1,2008,Belgium,673.0,KRC Genk,GEN
2,47,1,1,2008/2009,14,2008-11-29,492651,9984,3,2,2008,Belgium,673.0,KRC Genk,GEN
3,65,1,1,2008/2009,16,2008-12-13,492713,9986,1,0,2008,Belgium,673.0,KRC Genk,GEN
4,94,1,1,2008/2009,19,2009-01-24,492805,9998,2,0,2009,Belgium,673.0,KRC Genk,GEN


In [52]:
df_match = df_match.merge(df_team, how='left', left_on='away_team_api_id', right_on='team_api_id')
df_match.rename(columns={'team_long_name': 'away_team_name', 'id_x' : 'id'}, inplace=True)

df_match.drop(columns=['team_api_id', 'away_team_api_id','team_fifa_api_id_x', 'team_fifa_api_id_y','team_fifa_api_id_y','team_short_name_y','id_y'], axis=1, inplace=True)

df_match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_goal,away_team_goal,season_year,country_name,home_team_name,team_short_name_x,away_team_name
0,1,1,1,2008/2009,1,2008-08-17,492473,1,1,2008,Belgium,KRC Genk,GEN,Beerschot AC
1,29,1,1,2008/2009,12,2008-11-15,492583,1,1,2008,Belgium,KRC Genk,GEN,KSV Roeselare
2,47,1,1,2008/2009,14,2008-11-29,492651,3,2,2008,Belgium,KRC Genk,GEN,KSV Cercle Brugge
3,65,1,1,2008/2009,16,2008-12-13,492713,1,0,2008,Belgium,KRC Genk,GEN,Sporting Charleroi
4,94,1,1,2008/2009,19,2009-01-24,492805,2,0,2009,Belgium,KRC Genk,GEN,RAEC Mons


Thông tin dataframe sau khi kết

In [53]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25979 entries, 0 to 25978
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 25979 non-null  int64         
 1   country_id         25979 non-null  int64         
 2   league_id          25979 non-null  int64         
 3   season             25979 non-null  object        
 4   stage              25979 non-null  int64         
 5   date               25979 non-null  datetime64[ns]
 6   match_api_id       25979 non-null  int64         
 7   home_team_goal     25979 non-null  int64         
 8   away_team_goal     25979 non-null  int64         
 9   season_year        25979 non-null  int64         
 10  country_name       25979 non-null  object        
 11  home_team_name     25979 non-null  object        
 12  team_short_name_x  25979 non-null  object        
 13  away_team_name     25979 non-null  object        
dtypes: dat

### Kết bảng giải đấu

Kết bảng trận đấu với bảng giải đấu thông qua khóa `league_id`

In [54]:
df_match = df_match.merge(df_league, how='inner', left_on='league_id', right_on='id')

df_match.drop(columns=["id_y", "country_id_y" , "country_id_x"], inplace=True)
df_match.rename(columns={'id_x' : 'id'}, inplace=True)

df_match.head()

,id,league_id,season,stage,date,match_api_id,home_team_goal,away_team_goal,season_year,country_name,home_team_name,team_short_name_x,away_team_name,league_name
0,1,1,2008/2009,1,2008-08-17,492473,1,1,2008,Belgium,KRC Genk,GEN,Beerschot AC,Belgium Jupiler League
1,29,1,2008/2009,12,2008-11-15,492583,1,1,2008,Belgium,KRC Genk,GEN,KSV Roeselare,Belgium Jupiler League
2,47,1,2008/2009,14,2008-11-29,492651,3,2,2008,Belgium,KRC Genk,GEN,KSV Cercle Brugge,Belgium Jupiler League
3,65,1,2008/2009,16,2008-12-13,492713,1,0,2008,Belgium,KRC Genk,GEN,Sporting Charleroi,Belgium Jupiler League
4,94,1,2008/2009,19,2009-01-24,492805,2,0,2009,Belgium,KRC Genk,GEN,RAEC Mons,Belgium Jupiler League


Kiểm tra xem còn dữ liệu trống không

In [55]:
df_match.isna().sum().sum()

0

## Dữ liệu cầu thủ

Kết bảng dữ liệu cầu thủ với bảng đặc tính cầu thủ qua khóa `player_api_id`

In [56]:
df_player = df_player.merge(df_player_attribute, on=['player_api_id','player_fifa_api_id'], how = "inner")

df_player.rename(columns={'id_x' : 'id'}, inplace=True)

df_player.drop(columns=["id_y","player_api_id" , "player_fifa_api_id"], inplace=True)

df_player.head()

,id,player_name,birthday,height,weight,date,overall_rating,potential,preferred_foot,attacking_work_rate,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,2016-02-18 00:00:00,67.0,71.0,right,medium,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,1,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,2015-11-19 00:00:00,67.0,71.0,right,medium,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,1,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,2015-09-21 00:00:00,62.0,66.0,right,medium,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,1,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,2015-03-20 00:00:00,61.0,65.0,right,medium,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,1,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,2007-02-22 00:00:00,61.0,65.0,right,medium,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


Loại bỏ các dữ liệu trùng và dữ liệu trống

In [57]:
df_player.drop_duplicates(inplace = True)
df_player.dropna(inplace=True)

Kiểm tra còn dữ liệu trùng hoặc rỗng hay không

In [58]:
df_player.isna().sum().sum()

0

## Dữ liệu đội bóng

Như đã khảo sát bên trên, cột `buildUpPlayDribbling` được lược bỏ vì chứa nhiều giá trị rỗng

In [59]:
df_team_attribute.drop(columns=['buildUpPlayDribbling'], axis=1, inplace=True)
df_team_attribute.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,Little,50,Mixed,Organised,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,Normal,56,Mixed,Organised,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,Normal,54,Mixed,Organised,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,Little,70,Long,Organised,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,Little,52,Mixed,Organised,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


Kết bảng đội bóng và đặc tính đội bóng qua khóa `team_api_id`

In [60]:
df_team = df_team.merge(df_team_attribute , on = ['team_api_id','team_fifa_api_id'], how = 'inner')

df_team.drop(columns=["id_y","team_api_id" , "team_fifa_api_id","team_short_name"], inplace=True)

df_team.rename(columns={'id_x': 'id'}, inplace=True)

df_team.head()

,id,team_long_name,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,KRC Genk,2010-02-22 00:00:00,45,Balanced,Little,45,Mixed,Organised,50,...,60,Normal,Organised,70,High,65,Press,70,Wide,Cover
1,1,KRC Genk,2011-02-22 00:00:00,66,Balanced,Little,52,Mixed,Organised,65,...,51,Normal,Organised,48,Medium,47,Press,54,Normal,Offside Trap
2,1,KRC Genk,2012-02-22 00:00:00,53,Balanced,Little,55,Mixed,Organised,55,...,56,Normal,Organised,47,Medium,45,Press,55,Normal,Cover
3,1,KRC Genk,2013-09-20 00:00:00,58,Balanced,Little,38,Mixed,Organised,67,...,56,Normal,Organised,47,Medium,45,Press,55,Normal,Cover
4,1,KRC Genk,2014-09-19 00:00:00,58,Balanced,Normal,38,Mixed,Organised,67,...,56,Normal,Organised,47,Medium,45,Press,55,Normal,Cover


Thêm cột `year` để xác định thông tin năm của đội bóng

In [61]:
df_team['date'] = pd.to_datetime(df_team['date']) 
df_team['year'] = df_team['date'].dt.year

Lược bỏ dữ liệu trùng và rỗng

In [62]:
df_team.dropna(inplace=True)
df_team.drop_duplicates(inplace = True)

df_team.head()

,id,team_long_name,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,...,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,year
0,1,KRC Genk,2010-02-22,45,Balanced,Little,45,Mixed,Organised,50,...,Normal,Organised,70,High,65,Press,70,Wide,Cover,2010
1,1,KRC Genk,2011-02-22,66,Balanced,Little,52,Mixed,Organised,65,...,Normal,Organised,48,Medium,47,Press,54,Normal,Offside Trap,2011
2,1,KRC Genk,2012-02-22,53,Balanced,Little,55,Mixed,Organised,55,...,Normal,Organised,47,Medium,45,Press,55,Normal,Cover,2012
3,1,KRC Genk,2013-09-20,58,Balanced,Little,38,Mixed,Organised,67,...,Normal,Organised,47,Medium,45,Press,55,Normal,Cover,2013
4,1,KRC Genk,2014-09-19,58,Balanced,Normal,38,Mixed,Organised,67,...,Normal,Organised,47,Medium,45,Press,55,Normal,Cover,2014


Kiểm tra xem còn dữ liệu trùng hoặc rỗng không

In [63]:
df_team.isna().sum().sum()

0

# Phân tích dữ liệu

## Có bao nhiêu trận đấu trong mỗi giải đấu ở mùa giải 2016?

Để tìm được các trận đấu trong mỗi giải đấu, cần thực hiện các bước

1. Lọc ra toàn bộ các trận đấu trong mùa giải 2016
2. Nhóm các trận đấu theo giải đấu

In [64]:
match_2016 = df_match[df_match['season'] == '2015/2016']
match_2016.head()

,id,league_id,season,stage,date,match_api_id,home_team_goal,away_team_goal,season_year,country_name,home_team_name,team_short_name_x,away_team_name,league_name
91,1491,1,2015/2016,1,2015-07-25,1979834,3,1,2015,Belgium,KRC Genk,GEN,Oud-Heverlee Leuven,Belgium Jupiler League
92,1499,1,2015/2016,10,2015-10-04,1979898,3,1,2015,Belgium,KRC Genk,GEN,Standard de Liège,Belgium Jupiler League
93,1515,1,2015/2016,12,2015-10-23,1979914,0,4,2015,Belgium,KRC Genk,GEN,Royal Excel Mouscron,Belgium Jupiler League
94,1530,1,2015/2016,14,2015-10-30,1979929,0,2,2015,Belgium,KRC Genk,GEN,Sporting Lokeren,Belgium Jupiler League
95,1554,1,2015/2016,17,2015-11-28,1979953,0,1,2015,Belgium,KRC Genk,GEN,KAA Gent,Belgium Jupiler League


In [65]:
match_by_league = match_2016.groupby('league_name')['home_team_name'].count().sort_values()
match_by_league

league_name
Switzerland Super League    180
Scotland Premier League     228
Belgium Jupiler League      240
Poland Ekstraklasa          240
Germany 1. Bundesliga       306
Netherlands Eredivisie      306
Portugal Liga ZON Sagres    306
England Premier League      380
France Ligue 1              380
Italy Serie A               380
Spain LIGA BBVA             380
Name: home_team_name, dtype: int64

## Giải đấu nào có số trận hòa nhiều nhất trong mùa giải 2016?

Hàm `win` được xây dựng với mục tiêu xác định đội thắng trận ở mỗi trận đấu hoặc `DRAW` nếu hai đội hòa nhau

In [66]:
def win(df_match):
    home_score = df_match[0] #home_team_goal
    away_score = df_match[1] #away_team_goal
    home_team_name = df_match[2] #home_team_name
    away_team_name = df_match[3] #away_team_name
    
    if home_score > away_score:
        return home_team_name
    elif home_score < away_score:
        return away_team_name
    else:
        return 'DRAW'
     
df_match['winner'] = df_match[['home_team_goal', 'away_team_goal', 'home_team_name', 'away_team_name']].apply(win, axis=1)
df_match.head()

,id,league_id,season,stage,date,match_api_id,home_team_goal,away_team_goal,season_year,country_name,home_team_name,team_short_name_x,away_team_name,league_name,winner
0,1,1,2008/2009,1,2008-08-17,492473,1,1,2008,Belgium,KRC Genk,GEN,Beerschot AC,Belgium Jupiler League,DRAW
1,29,1,2008/2009,12,2008-11-15,492583,1,1,2008,Belgium,KRC Genk,GEN,KSV Roeselare,Belgium Jupiler League,DRAW
2,47,1,2008/2009,14,2008-11-29,492651,3,2,2008,Belgium,KRC Genk,GEN,KSV Cercle Brugge,Belgium Jupiler League,KRC Genk
3,65,1,2008/2009,16,2008-12-13,492713,1,0,2008,Belgium,KRC Genk,GEN,Sporting Charleroi,Belgium Jupiler League,KRC Genk
4,94,1,2008/2009,19,2009-01-24,492805,2,0,2009,Belgium,KRC Genk,GEN,RAEC Mons,Belgium Jupiler League,KRC Genk


Thống kê các giải đấu trong năm 2016 theo số trận hòa

In [67]:
match_2016 = df_match[df_match['season'] == '2015/2016']
match_2016[['winner',"league_name"]].value_counts().loc["DRAW"]

league_name
France Ligue 1              108
England Premier League      107
Italy Serie A                95
Spain LIGA BBVA              92
Portugal Liga ZON Sagres     76
Poland Ekstraklasa           74
Netherlands Eredivisie       74
Germany 1. Bundesliga        71
Belgium Jupiler League       59
Scotland Premier League      52
Switzerland Super League     47
dtype: int64

Vậy theo kết quả thống kê, giải **France Ligue 1** có số trận hòa nhiều nhất là **108** trận

## Giải đấu nào có số trận không hòa nhiều nhất trong mùa giải 2016?

Các trận đấu sẽ được khảo sát với 2 tiêu chí

1. Diễn ra ở mùa giải 2016
2. Có kết quả khác hòa

In [68]:
wins16 = match_2016.groupby(["league_name"]).apply(lambda x: (x["winner"]!= 'DRAW').sum()).reset_index(name='winner').sort_values('winner')
wins16

,league_name,winner
10,Switzerland Super League,133
6,Poland Ekstraklasa,166
8,Scotland Premier League,176
0,Belgium Jupiler League,181
7,Portugal Liga ZON Sagres,230
5,Netherlands Eredivisie,232
3,Germany 1. Bundesliga,235
2,France Ligue 1,272
1,England Premier League,273
4,Italy Serie A,285


Theo thống kê ta thấy được giải đấu **Spain LIGA BBVA** có số trận không hòa nhiều nhất **288** trận

## Đội bóng nào để thua ít trận nhất trong mùa giải 2016?

Hàm `loser` được thêm vào để xác định đội bóng thua trong mỗi trận

In [69]:
def lose(df_match):
    home_score = df_match[0] #home_team_goal
    away_score = df_match[1] #away_team_goal
    home_team_name = df_match[2] #home_team_name
    away_team_name = df_match[3] #away_team_name
    
    if home_score < away_score:
        return home_team_name
    elif home_score > away_score:
        return away_team_name
    else:
        return 'DRAW'
    
df_match['loser']  = df_match[['home_team_goal', 'away_team_goal', 'home_team_name', 'away_team_name']].apply(lose, axis=1)

In [70]:
match_2016 = df_match[df_match['season'] == '2015/2016']

Đội bóng thua ít trận nhất được xác định bằng cách đếm số trận thua

In [71]:
match_2016['loser'].value_counts().tail(10)

Borussia Dortmund      4
Pogoń Szczecin         4
Legia Warszawa         4
Roma                   4
Leicester City         3
Paris Saint-Germain    2
FC Bayern Munich       2
Sporting CP            2
Ajax                   2
PSV                    2
Name: loser, dtype: int64

Vậy theo kết quả thống kê, các đội bóng

* Paris Saint-Germain
* FC Bayern Munich
* Sporting CP
* Ajax
* PSV

Là các đội bóng ít thua nhất với số trận thua đều là **2** trận

## Đội bóng nào có số trận thắng nhiều nhất trong mùa giải 2016?

Đội bóng thắng nhiều nhất được tính bằng cách đếm số trận thắng trong mùa giải

In [72]:
match_2016['winner'].value_counts().head(10)

DRAW                   855
Paris Saint-Germain     30
Juventus                29
SL Benfica              29
FC Barcelona            29
Real Madrid CF          28
Atlético Madrid         28
FC Bayern Munich        28
Sporting CP             27
Celtic                  26
Name: winner, dtype: int64

Dựa theo kết quả thống kê, **Paris Saint-Germain** là đội bóng có số trận thắng nhiều nhất là **30** trận

## Giải đấu nào có nhiều bàn thắng nhất trong mùa giải 2016?

Tổng số bàn thắng của mỗi giải đấu được tính bằng tổng số bàn thắng của đội nhà và đội khách trong mỗi trận như sau

In [73]:
goals_2016 = match_2016.groupby('league_name')['home_team_goal'].sum().sort_values(ascending = False) + match_2016.groupby('league_name')['away_team_goal'].sum().sort_values(ascending = False)
goals_2016.sort_values(ascending = True, inplace =True)
goals_2016

league_name
Switzerland Super League     566
Poland Ekstraklasa           635
Scotland Premier League      650
Belgium Jupiler League       694
Portugal Liga ZON Sagres     831
Germany 1. Bundesliga        866
Netherlands Eredivisie       912
France Ligue 1               960
Italy Serie A                979
England Premier League      1026
Spain LIGA BBVA             1043
dtype: int64

Dựa theo kết quả thống kê, giải đấu **Spain LIGA BBVA** có số lượng bàn thắng nhiều nhất là **1043** bàn

## Đội bóng nào có kết quả cải thiện tốt nhất qua thời gian?

Việc đánh giá kết quả cải thiện của một đội bóng qua thời gian được thực hiện bằng cách so sánh kết quả của đội bóng đó trên 2 mốc thời gian - trong trường hợp này **mốc thời gian 2010 và 2016** được lựa chọn.

Tiêu chí đánh giá là tỷ lệ số trận thắng / thua giữa 2 giai đoạn

In [74]:
W2010 = df_match[(df_match['season_year'] == 2010) & (df_match['winner'] != 'DRAW')]                                                
L2010 = df_match[(df_match['season_year'] == 2010) & (df_match['loser'] != 'DRAW')]

countW2010 = W2010['winner'].count()
countL2010 = L2010['loser'].count()                                                    

R2010 = W2010['winner'].value_counts()/countW2010 - L2010['loser'].value_counts()/countL2010
R2010

1. FC Kaiserslautern      -0.000830
1. FC Köln                -0.003321
1. FC Nürnberg            -0.001660
1. FSV Mainz 05            0.002076
AC Arles-Avignon          -0.004981
                             ...   
Wisła Kraków               0.002906
Wolverhampton Wanderers   -0.003736
Xerez Club Deportivo      -0.001245
Zagłębie Lubin             0.000830
Śląsk Wrocław             -0.000830
Length: 208, dtype: float64

In [75]:
W2016 = df_match[(df_match['season_year'] == 2016) & (df_match['winner'] != 'DRAW')]                                                
L2016 = df_match[(df_match['season_year'] == 2016) & (df_match['loser'] != 'DRAW')]

countW2016 = W2016['winner'].count()
countL2016 = L2016['loser'].count()                                                    

R2016 = W2016['winner'].value_counts()/countW2016 - L2016['loser'].value_counts()/countL2016
R2016

1. FC Köln         -0.001641
1. FSV Mainz 05     0.001641
ADO Den Haag        0.000000
AS Monaco           0.004102
AS Saint-Étienne    0.001641
                      ...   
West Ham United     0.004102
Willem II          -0.006563
Wisła Kraków        0.000820
Zagłębie Lubin      0.003281
Śląsk Wrocław       0.000000
Length: 188, dtype: float64

Tìm ra các đội bóng có tỷ lệ thay đổi nhiều nhất trong khoản thời gian 2010 và 2016

In [76]:
(R2016 - R2010).dropna().sort_values().tail(10)

OGC Nice                    0.005347
Borussia Mönchengladbach    0.005802
Aberdeen                    0.006622
Sporting Charleroi          0.006662
SL Benfica                  0.006879
West Ham United             0.008668
Sporting CP                 0.009399
Atlético Madrid             0.009834
Paris Saint-Germain         0.010259
Juventus                    0.013521
dtype: float64

Dựa theo kết quả thống kê, **Juventus** là đội bóng có kết quả cải thiện tốt nhất trong giai đoạn 2010 đến 2016

## Cầu thủ nào sút phạt nhiều nhất?

In [77]:
most_penalties = df_player.groupby(['player_name'])['penalties'].max().sort_values(ascending=False)
most_penalties.head(10)

player_name
Rickie Lambert     96.0
Andrea Pirlo       95.0
Xavi Hernandez     95.0
Paul Scholes       95.0
Mario Balotelli    95.0
Adrian Mutu        94.0
Cesc Fabregas      94.0
Iker Casillas      94.0
David Trezeguet    94.0
Fernando Torres    93.0
Name: penalties, dtype: float64

Theo kết quả thống kê, **Rickie Lambert** là cầu thủ có số lần sút phạt nhiều nhất với **96** lần

## Có bao nhiêu cầu thủ có hiệu suất tổng thể hơn 90?

In [78]:
df_player[(df_player['overall_rating'] > 90)].player_name.nunique()

12

Theo kết quả thống kê, có **12** cầu thủ có hiệu suất tổng thể lớn hơn 90. Cụ thể danh sách các cầu thủ này gồm

In [79]:
df_player[(df_player['overall_rating'] > 90)].player_name.unique()

array(['Alessandro Nesta', 'Andres Iniesta', 'Cristiano Ronaldo',
       'Gianluigi Buffon', 'Gregory Coupet', 'Iker Casillas',
       'John Terry', 'Lionel Messi', 'Ronaldinho', 'Thierry Henry',
       'Wayne Rooney', 'Xavi Hernandez'], dtype=object)